In [1]:
import os
import pandas
import numpy
import cv2
# 2016-01-01で学習2016-01-02のprofitの正負を予測


def load_csv():
    df = pandas.read_csv("data/csv/0.csv")
    return df

def load_img():
    img_data = {}
    filename_list = os.listdir("data/img")
    for filename in filename_list:
        img = cv2.imread("data/img/{}".format(filename))
        ymd = filename.split(".")[0]
        img_data[ymd] = img
    return img_data

In [2]:
csv = load_csv()
img = load_img()

In [3]:
csv["nextday_profit"] = (csv["profit"].shift(-1)).apply(lambda x: 1 if x >= 0 else 0)
#csv["label_0"] = csv["nextday_profit"].apply(lambda x: 1 if x == 0 else 0)
#csv["label_1"] = csv["nextday_profit"].apply(lambda x: 1 if x == 1 else 0)

csv["label_0"] = csv["f1"].apply(lambda x: 1 if x <=0.5 else 0)
csv["label_1"] = csv["f1"].apply(lambda x: 1 if x >0.5 else 0)

In [5]:
split_ymd = "2016-10-30"


In [6]:
train_ymd = {ymd for ymd in img.keys() if ymd <= split_ymd}
test_ymd = {ymd for ymd in img.keys() if ymd > split_ymd}

In [7]:
tmp = pandas.DataFrame(train_ymd, columns=["date"])
train_img = numpy.array([arr for ymd, arr in img.items() if ymd <= split_ymd])
train_df = csv.merge(tmp, on="date", how="inner")
train_csv = train_df[["f1", "f2", "f3"]]
train_label = train_df[["label_0", "label_1"]]

tmp = pandas.DataFrame(test_ymd, columns=["date"])
test_img = numpy.array([arr for ymd, arr in img.items() if ymd > split_ymd])
test_df = csv.merge(tmp, on="date", how="inner")
test_csv = test_df[["f1", "f2", "f3"]]
test_label = test_df[["label_0", "label_1"]]

In [8]:
import keras
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Concatenate
from keras.layers.normalization import BatchNormalization
from keras import Input, Sequential
from keras.optimizers import SGD
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


In [9]:
length, height, width, channel = train_img.shape

input_img = Input(name="input_img", shape=(height, width, channel))
n_img = Conv2D(filters=32, kernel_size=(3, 3), padding="same",
                      activation="relu", name="img_conv2d_1")(input_img)
n_img = MaxPooling2D(pool_size=(2, 2), name="img_maxpool_1")(n_img)
n_img = Conv2D(filters=32, kernel_size=(3, 3), padding="same",
                      activation="relu", name="img_conv2d_2")(n_img)
n_img = BatchNormalization(name="img_bn_1")(n_img)
n_img = MaxPooling2D(pool_size=(2, 2), name="img_maxpool_2")(n_img)
n_img = Flatten(name="img_flatten_1")(n_img)
n_img = Dense(8, activation="relu", name="img_dence_1")(n_img)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
input_csv = Input(name="input_csv", shape=(3, ))
n_csv = Dense(8, activation="relu", name="csv_dence_1")(input_csv)

In [16]:
network = Concatenate()([n_img, n_csv])
network = Dense(8, activation="relu", name="merge_dense_1")(network)
network = BatchNormalization(name="merge_bn_1")(network)
network = Dense(2, activation="softmax", name="output")(network)

In [17]:
model = keras.Model(inputs=[input_img, input_csv], outputs=network)

In [18]:
model.compile(SGD(), loss=categorical_crossentropy, metrics=["accuracy"])

In [19]:
history = model.fit(
    [train_img, numpy.array(train_csv)], numpy.array(train_label),
    epochs=10,
    batch_size=16,
    validation_data=([test_img, numpy.array(test_csv)], numpy.array(test_label))
)

Train on 202 samples, validate on 42 samples
Epoch 1/10
202/202 [==============================] - 17s 83ms/step - loss: 1.0881 - acc: 0.4653 - val_loss: 5.2499 - val_acc: 0.3810
Epoch 2/10
202/202 [==============================] - 15s 75ms/step - loss: 0.5369 - acc: 0.7574 - val_loss: 6.2198 - val_acc: 0.3810
Epoch 3/10
202/202 [==============================] - 15s 75ms/step - loss: 0.2631 - acc: 0.9505 - val_loss: 4.7491 - val_acc: 0.3810
Epoch 4/10
202/202 [==============================] - 15s 73ms/step - loss: 0.1675 - acc: 0.9901 - val_loss: 3.9148 - val_acc: 0.3810
Epoch 5/10
202/202 [==============================] - 16s 77ms/step - loss: 0.1602 - acc: 0.9901 - val_loss: 3.1620 - val_acc: 0.3810
Epoch 6/10
202/202 [==============================] - 15s 76ms/step - loss: 0.1180 - acc: 1.0000 - val_loss: 2.7119 - val_acc: 0.3810
Epoch 7/10
202/202 [==============================] - 16s 77ms/step - loss: 0.1064 - acc: 1.0000 - val_loss: 2.3072 - val_acc: 0.3810
Epoch 8/10
202/20